In [1]:
import pandas as pd
import json
from datetime import datetime

In [2]:
data = pd.read_json('meeting.txt')

In [3]:
data.head()

,Company,city,country,is_host,jointime,leavetime,meetingid,userid
0,ABC,Baltimore,UNITED STATES,True,2017-04-01 00:00:01,2017-04-01 03:00:22,7f42225584,9FE1DE67F2
1,ABC,Baltimore,UNITED STATES,False,2017-04-01 00:00:04,2017-04-01 03:00:21,7f42225584,DEAB0BEF18
2,ABC,Boise City,UNITED STATES,False,2017-04-01 00:00:03,2017-04-01 03:00:19,7f42225584,AD1BB4E656
3,BCD,San Bernardino,UNITED STATES,False,2017-04-01 00:01:42,2017-04-01 00:13:07,5494d16309,2C1A95AC8C
4,BCD,Akron,UNITED STATES,True,2017-04-01 00:01:42,2017-04-01 00:13:42,5494d16309,0B781FA334


In [4]:
date_format = '%Y-%m-%d %H:%M:%S'

In [5]:
only_date_format = '%Y-%m-%d'

In [6]:
str_2_date_fn = lambda x: datetime.strptime(x, date_format)

In [7]:
data['jointime'] = data['jointime'].apply(str_2_date_fn)

In [8]:
data['leavetime'] = data['leavetime'].apply(str_2_date_fn)

In [9]:
data['date'] = data['leavetime'].apply(lambda x: x.strftime(only_date_format))

In [10]:
data['duration'] = data['leavetime'] - data['jointime']

In [11]:
data['duration_secs'] = data['duration'].dt.total_seconds()

In [13]:
data['duration_mins'] = data['duration'].dt.total_seconds().div(60)

In [229]:
data.head()

,Company,city,country,is_host,jointime,leavetime,meetingid,userid,date,duration,duration_secs,duration_mins
0,ABC,Baltimore,UNITED STATES,True,2017-04-01 00:00:01,2017-04-01 03:00:22,7f42225584,9FE1DE67F2,2017-04-01,03:00:21,10821.0,180.350000
1,ABC,Baltimore,UNITED STATES,False,2017-04-01 00:00:04,2017-04-01 03:00:21,7f42225584,DEAB0BEF18,2017-04-01,03:00:17,10817.0,180.283333
2,ABC,Boise City,UNITED STATES,False,2017-04-01 00:00:03,2017-04-01 03:00:19,7f42225584,AD1BB4E656,2017-04-01,03:00:16,10816.0,180.266667
3,BCD,San Bernardino,UNITED STATES,False,2017-04-01 00:01:42,2017-04-01 00:13:07,5494d16309,2C1A95AC8C,2017-04-01,00:11:25,685.0,11.416667
4,BCD,Akron,UNITED STATES,True,2017-04-01 00:01:42,2017-04-01 00:13:42,5494d16309,0B781FA334,2017-04-01,00:12:00,720.0,12.000000


In [127]:
company_list = data['Company'].unique()

In [128]:
company_date = data[['Company', 'date']].drop_duplicates()['Company'].value_counts()

In [129]:
total_meetings_7day_frame = company_date.rename_axis('Company').reset_index(name='total_meetings_7day')

In [130]:
total_meetings_7day_frame

,Company,total_meetings_7day
0,ABC,3
1,BCD,2


In [131]:
avg_meeting_sec_frame = data.groupby('Company').duration_secs.mean().rename_axis('Company').reset_index(name='avg_meeting_sec')

In [132]:
avg_meeting_sec_frame

,Company,avg_meeting_sec
0,ABC,4862.615385
1,BCD,1483.285714


In [133]:
avg_meeting_mins_frame = data.groupby('Company').duration_mins.mean().rename_axis('Company').reset_index(name='avg_meeting_mins')

In [134]:
avg_meeting_mins_frame

,Company,avg_meeting_mins
0,ABC,81.043590
1,BCD,24.721429


In [135]:
days_secs_frame = pd.merge(total_meetings_7day_frame, avg_meeting_sec_frame)

In [136]:
days_secs_frame

,Company,total_meetings_7day,avg_meeting_sec
0,ABC,3,4862.615385
1,BCD,2,1483.285714


In [202]:
country_frame = data[['Company','country']].drop_duplicates()

In [204]:
country_frame

,Company,country
0,ABC,UNITED STATES
3,BCD,UNITED STATES


In [205]:
days_secs_mins_frame = pd.merge(days_secs_frame, avg_meeting_mins_frame)

In [211]:
res = pd.merge(days_secs_mins_frame, country_frame)

In [212]:
res

,Company,total_meetings_7day,avg_meeting_sec,avg_meeting_mins,country
0,ABC,3,4862.615385,81.043590,UNITED STATES
1,BCD,2,1483.285714,24.721429,UNITED STATES


In [213]:
output  = res.to_json(orient='records')

In [214]:
import json

In [215]:
with open('output1_test2.txt', 'w') as outfile:
    json.dump(output, outfile, indent=4)